In [1]:
import requests as req
from bs4 import BeautifulSoup as bs4
import pandas as pd
import numpy as np
import time
import random
import matplotlib.pyplot as plt
import seaborn as sns
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementNotInteractableException
from webdriver_manager.chrome import ChromeDriverManager

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression as LinReg, Lasso, Ridge, ElasticNet
from sklearn.preprocessing import LabelEncoder, StandardScaler

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from fake_useragent import UserAgent


Extraemos la info de la URL 

In [2]:
años = ['2015','2016','2017','2018','2019','2020','2021','2022','2023']

for fecha in años:
    
    url = f"https://www.soccerstats.com/leagueview.asp?league=cleague_{fecha}"
    
    response = req.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        
        table = soup.find('table', {'id': 'btable'}) # Encuentra la tabla con el id "btable"


        if table:        # Verifica si la tabla fue encontrada
            
            rows = table.find_all('tr') # Extrae las filas de la tabla

            
            data = []
            for row in rows: # Extrae el contenido de las celdas y crea una lista de listas
                row_data = [cell.text.strip().replace('\n','') for cell in row.find_all(['td', 'th'])]
                data.append(row_data)

            globals()[f"df1_{fecha}"] = pd.DataFrame(data)    # Convierte la lista de listas en un DataFrame

        else:
            print("No se encontró la tabla con id 'btable' en la página.")
    else:
        print(f"Error al acceder a la página. Código de estado: {response.status_code}")

for fecha in años:
    url = f"https://www.soccerstats.com/leagueview.asp?league=cleague_{fecha}"
    response = req.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encuentra la tabla con el id "btable" y otros atributos
        table = soup.find('table', {'id': 'btable', 'cellpadding': '2', 'cellspacing': '0', 'width': '100%'})

        # Verifica si la tabla fue encontrada
        if table:
            # Convierte la tabla en un DataFrame
            globals()[f"df2_{fecha}"]  = pd.read_html(str(table))[0]

        else:
            print("No se encontró la tabla con los atributos específicos.")
    else:
        print(f"Error al acceder a la página. Código de estado: {response.status_code}")
for fecha in años: 
    url = f"https://www.soccerstats.com/leagueview.asp?league=cleague_{fecha}"
    response = req.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encuentra la tabla con el id "btable" y la clase "sortable"
        table = soup.find('table', {'id': 'btable', 'class': 'sortable'})

        # Verifica si la tabla fue encontrada
        if table:
            # Convierte la tabla en un DataFrame
            globals()[f"df3_{fecha}"] = pd.read_html(str(table))[0]

        
        else:
            print("No se encontró la tabla con los atributos específicos.")
    else:
        print(f"Error al acceder a la página. Código de estado: {response.status_code}")

C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\3418244951.py:46: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  globals()[f"df2_{fecha}"]  = pd.read_html(str(table))[0]
C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\3418244951.py:46: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  globals()[f"df2_{fecha}"]  = pd.read_html(str(table))[0]
C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\3418244951.py:46: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  globals()[f"df2_{fecha}"]  = pd.read_html(str(table))[0]
C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\3418244951.py:46: FutureWarning: Passing literal html to 'read

Limpiamos los Data Frames creados con la info extraida de la URL

In [3]:
clean_df = ['2015','2016','2017','2018','2019','2020']

for fecha in clean_df:
        # Procesamiento para df1
        globals()[f"df1_{fecha}"].drop(columns=2, inplace=True)
        globals()[f"df1_{fecha}"].fillna('-', inplace=True)
        globals()[f"df1_{fecha}"].columns = ['Fecha', 'Equipo1', 'Equipo2']

        # Procesamiento para df2
        globals()[f"df2_{fecha}"].drop(columns=['Unnamed: 0', 'Form'], inplace=True)

        # Procesamiento para df3
        globals()[f"df3_{fecha}"].drop(columns=['Unnamed: 11_level_0', 'Unnamed: 12_level_0'], inplace=True)
        new_columns = ['Matches of...', 'GP', 'Avg', '0.5+', '1.5+', '2.5+', '3.5+', '4.5+', 'BTS', 'CS', 'FTS']
        globals()[f"df3_{fecha}"].columns = new_columns
        globals()[f"df3_{fecha}"] = globals()[f"df3_{fecha}"].drop(32)


C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\482786782.py:13: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  globals()[f"df3_{fecha}"].drop(columns=['Unnamed: 11_level_0', 'Unnamed: 12_level_0'], inplace=True)
C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\482786782.py:13: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  globals()[f"df3_{fecha}"].drop(columns=['Unnamed: 11_level_0', 'Unnamed: 12_level_0'], inplace=True)
C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\482786782.py:13: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  globals()[f"df3_{fecha}"].drop(columns=['Unnamed: 11_level_0', 'Unnamed: 12_level_0'], inplace=True)
C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\482786782.py:13: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may imp

In [13]:
clean_df1 = ['2021']

for fecha in clean_df1:
        # Procesamiento para df1
        # Para eliminar las columnas 2 y 4 del DataFrame df1_{fecha}
        columns_to_drop = [2, 4]
        globals()[f"df1_{fecha}"].drop(columns=columns_to_drop, inplace=True)
        globals()[f"df1_{fecha}"].fillna('-', inplace=True)
        globals()[f"df1_{fecha}"].columns = ['Fecha', 'Equipo1', 'Equipo2']

        # Procesamiento para df2
        globals()[f"df2_{fecha}"].drop(columns=['Unnamed: 0', 'Form'], inplace=True)

        # Procesamiento para df3
        globals()[f"df3_{fecha}"].drop(columns=['Unnamed: 11_level_0', 'Unnamed: 12_level_0'], inplace=True)
        new_columns = ['Matches of...', 'GP', 'Avg', '0.5+', '1.5+', '2.5+', '3.5+', '4.5+', 'BTS', 'CS', 'FTS']
        globals()[f"df3_{fecha}"].columns = new_columns
        globals()[f"df3_{fecha}"] = globals()[f"df3_{fecha}"].drop(32)

C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\2691892414.py:15: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  globals()[f"df3_{fecha}"].drop(columns=['Unnamed: 11_level_0', 'Unnamed: 12_level_0'], inplace=True)


In [14]:
clean_df2 = ['2022', '2023']

for fecha in clean_df2:
    columns_to_drop = [2, 4]

    # Procesamiento para df1
    globals()[f"df1_{fecha}"].drop(columns=columns_to_drop, inplace=True)
    globals()[f"df1_{fecha}"].fillna('-', inplace=True)
    globals()[f"df1_{fecha}"].columns = ['Fecha', 'Equipo1', 'Equipo2']

    # Procesamiento para df2
    globals()[f"df2_{fecha}"].columns = globals()[f"df2_{fecha}"].iloc[0]
    globals()[f"df2_{fecha}"] = globals()[f"df2_{fecha}"][1:]
    globals()[f"df2_{fecha}"].drop(columns=['Form'], inplace=True)
    globals()[f"df2_{fecha}"].drop(globals()[f"df2_{fecha}"].columns[0], axis=1, inplace=True)

    # Procesamiento para df3
    unwanted_columns = [col for col in globals()[f"df3_{fecha}"].columns.levels[0] if 'Unnamed' in str(col)]
    globals()[f"df3_{fecha}"].drop(columns=unwanted_columns, inplace=True)

    new_columns = ['Matches of...', 'GP', 'Avg', '0.5+', '1.5+', '2.5+', '3.5+', '4.5+', 'BTS', 'CS', 'FTS']
    globals()[f"df3_{fecha}"].columns = new_columns
    globals()[f"df3_{fecha}"] = globals()[f"df3_{fecha}"].drop(32)

C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\3724687149.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[f"df2_{fecha}"].drop(columns=['Form'], inplace=True)
C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\3724687149.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()[f"df2_{fecha}"].drop(globals()[f"df2_{fecha}"].columns[0], axis=1, inplace=True)
C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\3724687149.py:19: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  globals()[f"df3_{fecha}"].drop(columns=unwanted_columns, inplace

-----------------------------------------


2024!!!! 



In [17]:
url = "https://www.soccerstats.com/leagueview.asp?league=cleague"
response = req.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encuentra la tabla con el id "btable"
    table = soup.find('table', {'id': 'btable'})

    # Verifica si la tabla fue encontrada
    if table:
        # Extrae las filas de la tabla
        rows = table.find_all('tr')

        # Extrae el contenido de las celdas y crea una lista de listas
        data = []
        for row in rows:
            row_data = [cell.text.strip().replace('\n', '') for cell in row.find_all(['td', 'th'])]
            data.append(row_data)

        # Convierte la lista de listas en un DataFrame
        df1_2024 = pd.DataFrame(data)
    else:
        print("No se encontró la tabla con id 'btable' en la página.")
else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

# Bloque 2
url = "https://www.soccerstats.com/leagueview.asp?league=cleague"
response = req.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encuentra la tabla con el id "btable" y otros atributos
    table = soup.find('table', {'id': 'btable', 'cellpadding': '2', 'cellspacing': '0', 'width': '100%'})

    # Verifica si la tabla fue encontrada
    if table:
        # Convierte la tabla en un DataFrame
        df2_2024 = pd.read_html(str(table))[0]
    else:
        print("No se encontró la tabla con los atributos específicos.")
else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")

# Bloque 3
url = f"https://www.soccerstats.com/leagueview.asp?league=cleague"
response = req.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encuentra la tabla con el id "btable" y la clase "sortable"
    table = soup.find('table', {'id': 'btable', 'class': 'sortable'})

    # Verifica si la tabla fue encontrada
    if table:
        # Convierte la tabla en un DataFrame
        df3_2024 = pd.read_html(str(table))[0]
    else:
        print("No se encontró la tabla con los atributos específicos.")
else:
    print(f"Error al acceder a la página. Código de estado: {response.status_code}")


C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\108507143.py:41: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2_2024 = pd.read_html(str(table))[0]
C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\108507143.py:60: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df3_2024 = pd.read_html(str(table))[0]


In [18]:
clean_df2 = ['2024']

for fecha in clean_df2:
    columns_to_drop = [2, 4]

    # Procesamiento para df1
    globals()[f"df1_{fecha}"].drop(columns=columns_to_drop, inplace=True)
    globals()[f"df1_{fecha}"].fillna('-', inplace=True)
    globals()[f"df1_{fecha}"].columns = ['Fecha', 'Equipo1', 'Equipo2']

    # Procesamiento para df2
    globals()[f"df2_{fecha}"].columns = globals()[f"df2_{fecha}"].iloc[0]
    globals()[f"df2_{fecha}"] = globals()[f"df2_{fecha}"][1:]
    globals()[f"df2_{fecha}"].drop(columns=['Form'], inplace=True)
    globals()[f"df2_{fecha}"].drop(globals()[f"df2_{fecha}"].columns[0], axis=1, inplace=True)

    # Procesamiento para df3
    unwanted_columns = [col for col in globals()[f"df3_{fecha}"].columns.levels[0] if 'Unnamed' in str(col)]
    globals()[f"df3_{fecha}"].drop(columns=unwanted_columns, inplace=True)

    new_columns = ['Matches of...', 'GP', 'Avg', '0.5+', '1.5+', '2.5+', '3.5+', '4.5+', 'BTS', 'CS', 'FTS']
    globals()[f"df3_{fecha}"].columns = new_columns

C:\Users\Aleja\AppData\Local\Temp\ipykernel_15184\1589966319.py:19: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  globals()[f"df3_{fecha}"].drop(columns=unwanted_columns, inplace=True)


In [21]:
ruta = ['2015','2016', '2017', '2018', '2019', '2020','2021','2022','2023','2024']

for fecha in ruta:
    # Obtén el nombre del DataFrame dinámicamente
    nombre_df = f"df2_{fecha}"

    # Obtén el DataFrame correspondiente usando globals()
    df_actual = globals()[nombre_df]

    # Ruta de guardado
    ruta_guardado = fr"C:\Users\Aleja\OneDrive\Documentos\PROYECTO2\{nombre_df}.csv"

    # Guardar el DataFrame como archivo CSV en la ruta especificada
    df_actual.to_csv(ruta_guardado, index=False)


-----

Asi quedan los DataFrames

FASES DE LA CHAMPIONS Y QUE EQUIPOS ESTAN

In [22]:
df1_2016

,Fecha,Equipo1,Equipo2
0,8th Finals,-,-
1,16 Feb 16,Benfica,Zenit
2,16 Feb 16,Paris SG,Chelsea
3,17 Feb 16,AS Roma,Real Madrid
4,17 Feb 16,Gent,Wolfsburg
5,23 Feb 16,Arsenal,FC Barcelona
6,23 Feb 16,Juventus,Bayern Munich
7,24 Feb 16,Dynamo Kiev,Manchester C.
8,24 Feb 16,PSV Eindhoven,Atletico Mad.
9,8 Mar 16,Real Madrid,AS Roma


-----

TABLA ESTADISTICA POR EQUIPOS

In [197]:
df2_2016

,Unnamed: 1,GP,W,D,L,GF,GA,GD,Pts,PPG,CS,FTS
0,Real Madrid,13,9,3,1,28,6,22,30,2.31,77%,23%
1,Bayern Munich,12,8,2,2,30,11,19,26,2.17,42%,17%
2,FC Barcelona,10,7,2,1,22,8,14,23,2.30,30%,10%
3,Atletico Madrid,13,6,4,3,17,8,9,22,1.69,62%,23%
4,Manchester City,12,6,3,3,18,12,6,21,1.75,25%,33%
5,Wolfsburg,10,7,0,3,15,11,4,21,2.10,50%,20%
6,Paris SG,10,6,2,2,18,6,12,20,2.00,50%,30%
7,Benfica,10,5,2,3,15,12,3,17,1.70,20%,10%
8,Zenit,8,5,0,3,14,9,5,15,1.88,25%,12%
9,Chelsea,8,4,1,3,15,7,8,13,1.63,50%,12%



Significado :

GP: Partidos Jugados

W: Partidos Ganados

D: Partidos Empatados (Draw)

L: Partidos Perdidos

GF: Goles a Favor (Goals For)

GA: Goles en Contra (Goals Against)

GD: Diferencia de Goles (Goal Difference, calculada restando los goles en contra de los goles a favor)

Pts: Puntos acumulados en la liga (3 puntos por victoria, 1 punto por empate)

PPG: Puntos Promedio por Partido (calculado dividiendo los puntos entre los partidos jugados)

CS: Clean Sheets (Partidos sin encajar goles)

FTS: Failed to Score (Partidos sin anotar goles)

----

TABLA ESTADISTICAS DE GOLES POR EQUIPOS

In [46]:
df3_2015

,Matches of...,GP,Avg,0.5+,1.5+,2.5+,3.5+,4.5+,BTS,CS,FTS
0,BATE Borisov,6.0,4.33,100%,100%,83%,50%,50%,17%,0%,83%
1,Schalke 04,8.0,4.00,100%,88%,50%,50%,50%,62%,12%,25%
2,Sporting CP,6.0,4.00,100%,83%,67%,67%,33%,83%,0%,17%
3,Bayern Munich,12.0,3.83,92%,75%,67%,50%,42%,42%,50%,17%
4,Galatasaray,6.0,3.83,100%,100%,67%,67%,50%,67%,0%,33%
5,Olympiakos,6.0,3.83,100%,83%,67%,67%,50%,50%,17%,33%
6,FC Porto,10.0,3.70,100%,100%,70%,50%,20%,60%,40%,0%
7,AS Roma,6.0,3.67,100%,100%,33%,33%,33%,67%,0%,33%
8,Arsenal,8.0,3.63,100%,100%,62%,50%,38%,62%,25%,12%
9,Chelsea,8.0,3.25,100%,88%,50%,50%,25%,62%,38%,0%


Significado :

GP: número de partidos jugados por el equipo 

Avg: promedio de goles en partidos jugados por el equipo (goles del equipo + goles del oponente)

0,5+: partidos con un número total de goles (goles del equipo + goles del oponente) superior a 0,5

1,5+: partidos con un número total de goles (goles del equipo + goles del oponente) superior a 1,5

2,5+: partidos con un número total de goles (goles del equipo + goles del oponente) superior a 2,5

3,5+: partidos con un número total de goles (goles del equipo + goles del oponente) superior a 3,5

4,5+: partidos con un número total de goles (goles del equipo + goles del oponente) superior a 4,5

BTS: partidos donde ambos equipos han marcado

CS: partidos donde el equipo no encajó ningún gol

FTS: partidos donde el equipo no anotó

In [23]:
lista = ['2015','2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
for num in lista:
    globals()[f"df1_{num}"] = pd.read_csv(r"C:\Users\Aleja\OneDrive\Documentos\PROYECTO2\df1_{}.csv".format(num))

In [ ]:
participantes = pd.read_csv(r"C:\Users\Aleja\OneDrive\Documentos\PROYECTO2\participantes.csv")

In [34]:
df_unido3.to_csv(r"C:\Users\Aleja\OneDrive\Documentos\PROYECTO2\df_unido3.csv",index=False)

In [33]:
df_unido3 = pd.concat([df3_2015,df3_2016,df3_2017,df3_2018,df3_2019
                      ,df3_2021,df3_2022,df3_2023],axis = 0)

In [29]:
df_unido

,Fecha,Equipo1,Equipo2
0,8th Finals,-,-
1,17 Feb 15,Paris SG,Chelsea
2,17 Feb 15,Shakhtar,Bayern Munich
3,18 Feb 15,Basel,FC Porto
4,18 Feb 15,Schalke 04,Real Madrid
...,...,...,...
28,10 May 23,AC Milan,Inter Milan
29,16 May 23,Inter Milan,AC Milan
30,17 May 23,Manchester City,Real Madrid
31,Final,-,-


In [26]:
df_unido['Equipo1'] = df_unido['Equipo1'].replace({
    'Manchester C.': 'Manchester City',
    'Atletico Mad.': 'Atletico Madrid',
    'Manchester Utd': 'Manchester U.'
})

In [27]:
df_unido['Equipo2'] = df_unido['Equipo2'].replace({
    'Manchester C.': 'Manchester City',
    'Atletico Mad.': 'Atletico Madrid',
    'Manchester Utd': 'Manchester U.',
    'PSV Eindhoven.*': 'PSV Eindhoven',
    'Juventus.*AET': 'Juventus',
    'Bayern Munich.*AET': 'Bayern Munich',
    'ChelseaFeatured': 'Chelsea',
    'Chelsea.*AET': 'Chelsea',
    'Real MadridFeatured': 'Real Madrid',
    'FC Porto.*AET': 'FC Porto',
    'Manchester C.*AET': 'Manchester City',
    'Atletico Mad.*AETp.*': 'Atletico Madrid',
    'Paris SG.*AET': 'Paris SG' 
}, regex=True)


----

Extraemo mas informacion

In [ ]:
# Lista de enlaces de equipos
id = ['18', '4', '7', '9', '32', '26', '16', '8', '28', '21', '31', '11', '19', '25', '13', '10']
enlaces_equipos = []

# Genera los enlaces para cada equipo
for x in id:
    enlace_equipo = f'https://www.soccerstats.com/leagueview_team.asp?league=cleague&team1id={x}'
    enlaces_equipos.append(enlace_equipo)

# Lista para almacenar los nombres de los equipos
nombres_equipos = []

# Lista para almacenar DataFrames de equipos
equipos_dataframes = []

# Itera sobre los enlaces y obtén el nombre y estadísticas de cada equipo
for enlace in enlaces_equipos:
    # Obtén el HTML de la página del equipo
    response = req.get(enlace)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encuentra todas las etiquetas h2
        h2_tags = soup.find_all('h2')

        # Verifica si hay al menos dos etiquetas h2
        if len(h2_tags) >= 2:
            # Extrae el texto de la segunda etiqueta h2
            equipo_nombre = h2_tags[1].get_text(strip=True)
            nombres_equipos.append(equipo_nombre)
        else:
            print(f"No se encontraron suficientes etiquetas h2 en la página: {enlace}")

        # Encuentra la tabla con los atributos específicos
        table = soup.find('table', {'cellspacing': '0', 'cellpadding': '1', 'width': '100%', 'style': 'margin-top:0px;'})
        
        # Verifica si la tabla fue encontrada
        if table:
            # Convierte la tabla en un DataFrame
            df_equipo = pd.read_html(str(table))[0]

            # Agrega el nombre del equipo como una nueva columna
            df_equipo['Equipo'] = equipo_nombre

            # Añade el DataFrame al listado
            equipos_dataframes.append(df_equipo)
        else:
            print(f"No se encontró la tabla con los atributos específicos en la página: {enlace}")
    else:
        print(f"Error al acceder a la página {enlace}. Código de estado: {response.status_code}")

# Concatena los DataFrames de estadísticas de cada equipo al DataFrame final
df_final = pd.concat(equipos_dataframes, ignore_index=True)

# Imprime el DataFrame final
df_final


In [187]:
df_final.to_csv(r"C:/Users/Aleja/OneDrive/Documentos/PROYECTO2/df_final.csv", index=False)

In [188]:
df_final = pd.read_csv(r"C:\Users\Aleja\OneDrive\Documentos\PROYECTO2\df_final.csv")

In [189]:
df_final

,H/A/T,Played,W,D,L,Equipos
0,Home,10,6,1,3,FC Kobenhavn
1,Away,10,6,2,2,FC Kobenhavn
2,Total,20,12,3,5,FC Kobenhavn
3,Home,12,8,2,2,RB Leipzig
4,Away,11,4,2,5,RB Leipzig
5,Total,23,12,4,7,RB Leipzig
6,Home,13,6,3,4,Lazio
7,Away,14,6,1,7,Lazio
8,Total,27,12,4,11,Lazio
9,Home,12,8,3,1,Paris SG
